# Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Are you ready to dive into the world of deep learning and artificial intelligence? Look no further than DeepLearning.AI. Founded by Andrew Ng, a prominent figure in the AI community, this platform offers top-notch courses to help you master the complexities of deep learning. With a focus on practical applications and hands-on projects, DeepLearning.AI equips you with the skills needed to thrive in this cutting-edge field. Whether you're a beginner or an experienced professional, there's something for everyone. Embark on your deep learning journey today with DeepLearning.AI!


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Are you ready to dive into the world of deep learning and artificial intelligence? Look no further than DeepLearning.AI. Founded by Andrew Ng, a prominent figure in the AI community, this platform offers top-notch courses to help you master the complexities of deep learning. With a focus on practical applications and hands-on projects, DeepLearning.AI equips you with the skills needed to thrive in this cutting-edge field. Whether you're a beginner or an experienced professional, there's something for everyone. Embark on your deep learning journey today with DeepLearning.AI!

--------------------------------------------------------------------------------
Critic (

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unleashing the Power of Deep Learning with DeepLearning.AI"

Are you ready to dive into the world of deep learning and artificial intelligence? Look no further than DeepLearning.AI. Founded by Andrew Ng, a prominent figure in the AI community, this platform offers top-notch courses to help you master the complexities of deep learning. With a focus on practical applications and hands-on projects, DeepLearning.AI equips you with the skills needed to thrive in this cutting-edge field. Whether you're a beginner or an experienced professional, there's something for everyone. Embark on your deep learning journey today with DeepLearning.AI!

--------------------------------------------------------------------------------

*******


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

After aggregating feedback from all reviewers, it is evident that there are several key areas for improvement in the writing related to deep learning courses and online education:

1. **SEO Enhancement**:
    - Incorporate specific keywords like 'deep learning courses,' 'Andrew Ng,' and 'artificial intelligence training' to improve search engine visibility.
    - Create a concise and compelling meta description that includes relevant keywords for better click-through rates.

2. **Legal Accuracy**:
    - Verify all references to Andrew Ng for accuracy and ensure no endorsement or affiliation is implied without permission.
    - Check for any potential trademark issues related to terms like 'deep learning' and 'artificial intelligence' in the content.

3. **Ethical Considerations**:
    - Ensure that all claims about DeepLearning.AI and its courses are accurate and supported by e

## Get the summary

In [15]:
print(res.summary)

Title: "Master Deep Learning Skills with Online Courses by DeepLearning.AI"

Are you looking to excel in deep learning and artificial intelligence training? Discover the world of DeepLearning.AI led by the renowned expert Andrew Ng. From foundational concepts to advanced applications, these deep learning courses offer hands-on experience to elevate your skills. Dive into projects and practical learning, suitable for beginners and seasoned professionals alike. Kickstart your deep learning journey today with DeepLearning.AI's comprehensive online resources. Learn from the best and embrace the future of AI education.
